# Média móvel de casos e mortes por Covid-19 no Brasil, Reino Unido e Estados Unidos

Os dados utilizados estão disponíveis em https://ourworldindata.org/coronavirus (acesso em 02/02/2022).

Referências:

1) https://stackoverflow.com/questions/9031783/hide-all-warnings-in-ipython

2) https://stackoverflow.com/questions/40060842/moving-average-pandas

3) https://pandas.pydata.org/docs/reference/api/pandas.concat.html

4) https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

5) https://plotly.com/python/figurewidget/

Roger Sampaio Bif

#### Importar as bibliotecas necessárias

In [1]:
import pandas as pd
import plotly.graph_objects as go
import warnings
from datetime import datetime
from ipywidgets import widgets

warnings.filterwarnings('ignore') # Ref 1

#### Primeiro tratamento de dados
##### Selecionar apenas informações dos países que estarão no dashboard
##### Reduzir a base de dados às colunas "iso_code", "date", "new_cases", "new_deaths"

In [2]:
df = pd.read_csv('covid_data.csv')

paises = ['BRA', 'GBR', 'USA']
df = df.loc[df.iso_code.isin(paises)]

df = df[["iso_code", "date", "new_cases", "new_deaths"]]
df = df.reset_index(drop=True)

#### Converter as datas para datetime.datetime

In [3]:
def para_date(linha):
    linha.date = datetime.strptime(linha.date, '%Y-%m-%d')
    return linha
df = df.apply(para_date, axis=1)
df.head()

,iso_code,date,new_cases,new_deaths
0,BRA,2020-02-26,1.0,NaN
1,BRA,2020-02-27,0.0,NaN
2,BRA,2020-02-28,0.0,NaN
3,BRA,2020-02-29,1.0,NaN
4,BRA,2020-03-01,0.0,NaN


#### Adicionar as colunas "movel_casos" e "movel_mortes", que representam a média móvel de 7 dias de casos e mortes

In [4]:
 def add_movel(paises, df):
    dfs = []
    for pais in paises:
        temp_df = df.loc[df.iso_code == pais]
        temp_df['movel_casos'] = df.new_cases.rolling(window=7).mean() # Ref 2
        temp_df['movel_mortes'] = df.new_deaths.rolling(window=7).mean()
        dfs.append(temp_df)
    df_final = pd.DataFrame()
    return pd.concat(dfs) # Ref 3

df = add_movel(paises, df)

"""
A função utilizada infelizmente utiliza os 7 ultimos resultados para os 7 primeiros dias disponíveis, então as médias móveis
de casos e mortes serão 0 nos 7 primeiros dias.
"""

temp_BRA = df.loc[df.iso_code == 'BRA']
temp_EUA = df.loc[df.iso_code == 'USA']
temp_UK = df.loc[df.iso_code == 'GBR']

temp_BRA['movel_casos'][:7] = 0
temp_BRA['movel_mortes'][:7] = 0
temp_EUA['movel_casos'][:7] = 0
temp_EUA['movel_mortes'][:7] = 0
temp_UK['movel_casos'][:7] = 0
temp_UK['movel_mortes'][:7] = 0

df = pd.concat([temp_BRA, temp_EUA, temp_UK])
display(df[100:105])

,iso_code,date,new_cases,new_deaths,movel_casos,movel_mortes
100,BRA,2020-06-05,32101.0,1076.0,26072.857143,1031.571429
101,BRA,2020-06-06,23790.0,801.0,24854.142857,1022.571429
102,BRA,2020-06-07,17988.0,476.0,25534.857143,1023.285714
103,BRA,2020-06-08,17338.0,863.0,25847.571429,1040.714286
104,BRA,2020-06-09,31698.0,1193.0,25850.571429,1016.142857


## Criar os widgets (ref 4)

#### Slider

In [5]:
options_slider = [str(m) + '/' + str(a) for a in range(2020,2022) for m in range(1,13)]  # ["1/2020",..."12/2021"]
options_slider = ['0' + data if int(data.split('/')[0]) < 10 else data for data in options_slider]  # ["01/2020", ... "12/2021"]
options_slider += ['01/2022', '02/2022']

index_slider = (0, len(options_slider) - 1)
slider_data = widgets.SelectionRangeSlider(options=options_slider, index=index_slider)


#### CheckBox

In [6]:
check_BRA = widgets.Checkbox(description='Brasil',value=True)
check_EUA = widgets.Checkbox(description='Estados Unidos',value=True)
check_UK  = widgets.Checkbox(description='Reino Unido',value=True)

#### Dropdown

In [7]:
dropdown = widgets.Dropdown(options=['Casos', 'Mortes'],
                            value='Casos',
                            description='Selecione',
                            disabled=False)

#### Gráfico (ref 5)

In [8]:
graf_BRA = go.Scatter(x=df.loc[df.iso_code == 'BRA'].date,
                       y=df.loc[df.iso_code == 'BRA'].movel_casos,
                       name='Brasil')

graf_EUA = go.Scatter(x=df.loc[df.iso_code == 'USA'].date,
                       y=df.loc[df.iso_code == 'USA'].movel_casos,
                       name='Estados Unidos')

graf_UK = go.Scatter(x=df.loc[df.iso_code == 'GBR'].date,
                       y=df.loc[df.iso_code == 'GBR'].movel_casos,
                       name='Reino Unido')

graf = go.FigureWidget(data=[graf_BRA, graf_UK, graf_EUA])

#graf.show()

#### Containers

In [9]:
container_check = widgets.HBox(children=[check_BRA, check_UK, check_EUA])
container_2 = widgets.HBox(children=[dropdown, slider_data])

## Funcionalidade 

In [10]:
def resposta(change):
    # Dropdown
    if dropdown.value == 'Casos':
        resp_drop = 'movel_casos'
    elif dropdown.value == 'Mortes':
        resp_drop = 'movel_mortes'
        
        
    # Slider
    data_inicio, data_fim = slider_data.value
    data_inicio = datetime.strptime(data_inicio, '%m/%Y')
    data_fim = datetime.strptime(data_fim, '%m/%Y')
    
    checkbox_list = [check_BRA, check_UK, check_EUA]
    for i in range(0,3):  # índices dos países quando foi definido o FigureWidget "graf"
        graf.data[i]['x'] = df.loc[(df.iso_code == paises[i]) & (df.date > data_inicio) & (df.date <= data_fim)].date
        if checkbox_list[i].value == True:
            graf.data[i]['y'] = df.loc[(df.iso_code == paises[i]) & (df.date > data_inicio) & (df.date <= data_fim)][resp_drop]
        else:
            graf.data[i]['y'] = []


check_BRA.observe(resposta, names='value')
check_EUA.observe(resposta, names='value')
check_UK.observe(resposta, names='value')
dropdown.observe(resposta, names='value')
slider_data.observe(resposta, names='value')



## Mostrar o gráfico

In [11]:
plot = widgets.VBox(children=[container_check, container_2, graf])
display(plot)